In [75]:
import pandas as pd

from baseball_scraper import playerid_lookup
from baseball_scraper import statcast_pitcher
from baseball_scraper import batting_stats_range
from baseball_scraper import statcast
from baseball_scraper import statcast_batter

from sklearn.cluster import KMeans, k_means
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

In [157]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 400)

In [77]:
#data = batting_stats_range('2015-04-01', '2020-10-27')


In [78]:
#data.head()

In [79]:
#data[data['G'] > 100]

In [80]:
# hitters = data[data['G'] > 100]

# hitter_ids = hitters['mlb_ID']

# hitter_ids.to_csv('./veteran_hitter_ids',index=False)

In [81]:
hitter_ids = pd.read_csv('../veteran_hitter_ids')
hitter_ids.drop(columns=['Unnamed: 0'], inplace=True)

In [82]:
hitter_ids

,mlb_ID
0,547989
1,554429
2,660670
3,542436
4,642715
...,...
689,458913
690,605548
691,475582
692,450314


In [83]:
# sc_data = statcast(start_dt='2015-04-01', end_dt= '2020-10-29')

In [84]:
#sc_data.to_csv('./statcast_data_2015.csv', index=False)

In [85]:
sc_data = pd.read_csv('../statcast_data_2015.csv')

/Users/johnnybex/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (45) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [86]:
sc_data['game_date'].isnull().sum()

0

In [87]:
sc_data.shape

(3925778, 91)

In [88]:
sc_data['pitcher'].isnull().sum()

0

In [89]:
#sc_data.dropna(subset=['batter'],inplace=True)

In [90]:
sc_data['batter'] = sc_data['batter'].astype(int)

In [91]:
sc_data

,Unnamed: 0,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,0,0,FF,2020-10-27,96.7,1.58,5.99,Julio Urias,642715,628711.0,strikeout,called_strike,NaN,NaN,NaN,NaN,4.0,Willy Adames called out on strikes.,W,R,L,LAD,TB,S,2.0,NaN,0.0,2.0,2020.0,0.18,1.63,-0.53,2.29,NaN,NaN,NaN,2.0,9.0,Top,NaN,NaN,NaN,NaN,605131.0,NaN,NaN,-5.950264,-140.490456,-7.897391,3.772000,32.321911,-8.981441,3.50,1.69,NaN,NaN,NaN,95.4,2615.0,5.7,635886.0,628711.0,605131.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,54.82,NaN,NaN,0.0,1.0,0.0,0.0,NaN,65.0,3.0,4-Seam Fastball,3.0,1.0,1.0,3.0,1.0,3.0,1.0,3.0,Standard,Standard
1,1,1,FF,2020-10-27,94.1,2.91,5.45,Julio Urias,642715,628711.0,NaN,called_strike,NaN,NaN,NaN,NaN,1.0,NaN,W,R,L,LAD,TB,S,NaN,NaN,0.0,1.0,2020.0,0.75,1.30,-0.55,3.03,NaN,NaN,NaN,2.0,9.0,Top,NaN,NaN,NaN,NaN,605131.0,NaN,NaN,-10.560246,-136.599519,-3.429867,11.723598,29.183810,-15.237217,3.50,1.69,NaN,NaN,NaN,93.4,2470.0,5.9,635886.0,628711.0,605131.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,54.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.0,2.0,4-Seam Fastball,3.0,1.0,1.0,3.0,1.0,3.0,1.0,3.0,Standard,Standard
2,2,2,FF,2020-10-27,94.9,1.77,6.02,Julio Urias,642715,628711.0,NaN,swinging_strike,NaN,NaN,NaN,NaN,2.0,NaN,W,R,L,LAD,TB,S,NaN,NaN,0.0,0.0,2020.0,0.23,1.47,-0.04,3.32,NaN,NaN,NaN,2.0,9.0,Top,NaN,NaN,NaN,NaN,605131.0,NaN,NaN,-5.199252,-138.098234,-4.637970,4.158758,30.838499,-12.535677,3.50,1.69,NaN,NaN,NaN,94.0,2397.0,5.7,635886.0,628711.0,605131.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,54.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.0,1.0,4-Seam Fastball,3.0,1.0,1.0,3.0,1.0,3.0,1.0,3.0,Standard,Standard
3,3,3,FF,2020-10-27,94.4,1.66,5.93,Julio Urias,670712,628711.0,strikeout,called_strike,NaN,NaN,NaN,NaN,4.0,Mike Brosseau called out on strikes.,W,R,L,LAD,TB,S,2.0,NaN,3.0,2.0,2020.0,0.27,1.50,-0.37,2.15,NaN,NaN,NaN,1.0,9.0,Top,NaN,NaN,NaN,NaN,605131.0,NaN,NaN,-5.843595,-137.294295,-7.414897,4.754147,30.016237,-11.778755,3.34,1.53,NaN,NaN,NaN,93.7,2508.0,5.9,635886.0,628711.0,605131.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,54.60,NaN,NaN,0.0,1.0,0.0,0.0,NaN,64.0,6.0,4-Seam Fastball,3.0,1.0,1.0,3.0,1.0,3.0,1.0,3.0,Standard,Standard
4,4,4,CU,2020-10-27,81.4,1.46,6.06,Julio Urias,670712,628711.0,NaN,ball,NaN,NaN,NaN,NaN,13.0,NaN,W,R,L,LAD,TB,B,NaN,NaN,2.0,2.0,2020.0,-1.67,-0.15,-0.14,0.96,NaN,NaN,NaN,1.0,9.0,Top,NaN,NaN,NaN,NaN,605131.0,NaN,NaN,-0.383207,-118.447810,-4.454166,-15.553576,24.451936,-32.892744,3.34,1.53,NaN,NaN,NaN,80.2,3031.0,5.7,635886.0,628711.0,605131.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,54.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.0,5.0,Curveball,3.0,1.0,1.0,3.0,1.0,3.0,1.0,3.0,Standard,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [92]:
# vet_hitter_statcast = sc_data[sc_data['batter'] in hitter_ids.values]
vet_hitter_statcast = pd.merge(sc_data, hitter_ids, left_on= 'batter', right_on='mlb_ID')

In [93]:
vet_hitter_statcast.loc[:, 'game_date'] = pd.to_datetime(vet_hitter_statcast['game_date'],
                                                        format= '%Y-%m-%d')

In [94]:
vet_hitter_statcast[vet_hitter_statcast['game_date'] > pd.to_datetime('2015-11-1')]

,Unnamed: 0,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,mlb_ID
0,0,0,FF,2020-10-27,96.7,1.58,5.99,Julio Urias,642715,628711.0,strikeout,called_strike,NaN,NaN,NaN,NaN,4.0,Willy Adames called out on strikes.,W,R,L,LAD,TB,S,2.0,NaN,0.0,2.0,2020.0,0.18,1.63,-0.53,2.29,NaN,NaN,NaN,2.0,9.0,Top,NaN,NaN,NaN,NaN,605131.0,NaN,NaN,-5.950264,-140.490456,-7.897391,3.772000,32.321911,-8.981441,3.50,1.69,NaN,NaN,NaN,95.4,2615.0,5.7,635886.0,628711.0,605131.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,54.82,NaN,NaN,0.0,1.0,0.0,0.0,NaN,65.0,3.0,4-Seam Fastball,3.0,1.0,1.0,3.0,1.0,3.0,1.0,3.0,Standard,Standard,642715
1,1,1,FF,2020-10-27,94.1,2.91,5.45,Julio Urias,642715,628711.0,NaN,called_strike,NaN,NaN,NaN,NaN,1.0,NaN,W,R,L,LAD,TB,S,NaN,NaN,0.0,1.0,2020.0,0.75,1.30,-0.55,3.03,NaN,NaN,NaN,2.0,9.0,Top,NaN,NaN,NaN,NaN,605131.0,NaN,NaN,-10.560246,-136.599519,-3.429867,11.723598,29.183810,-15.237217,3.50,1.69,NaN,NaN,NaN,93.4,2470.0,5.9,635886.0,628711.0,605131.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,54.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.0,2.0,4-Seam Fastball,3.0,1.0,1.0,3.0,1.0,3.0,1.0,3.0,Standard,Standard,642715
2,2,2,FF,2020-10-27,94.9,1.77,6.02,Julio Urias,642715,628711.0,NaN,swinging_strike,NaN,NaN,NaN,NaN,2.0,NaN,W,R,L,LAD,TB,S,NaN,NaN,0.0,0.0,2020.0,0.23,1.47,-0.04,3.32,NaN,NaN,NaN,2.0,9.0,Top,NaN,NaN,NaN,NaN,605131.0,NaN,NaN,-5.199252,-138.098234,-4.637970,4.158758,30.838499,-12.535677,3.50,1.69,NaN,NaN,NaN,94.0,2397.0,5.7,635886.0,628711.0,605131.0,571970.0,571771.0,621458.0,608369.0,621035.0,641355.0,605141.0,54.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.0,1.0,4-Seam Fastball,3.0,1.0,1.0,3.0,1.0,3.0,1.0,3.0,Standard,Standard,642715
3,68,68,SI,2020-10-27,102.1,-1.51,6.51,Brusdar Graterol,642715,660813.0,field_out,hit_into_play,NaN,NaN,NaN,NaN,4.0,Willy Adames grounds out to first baseman Max ...,W,R,R,LAD,TB,X,3.0,ground_ball,2.0,0.0,2020.0,-1.19,0.59,-0.74,2.78,NaN,NaN,NaN,0.0,7.0,Top,153.07,157.69,NaN,NaN,605131.0,NaN,NaN,4.949605,-148.533560,-6.597126,-18.791255,34.128576,-22.071340,3.50,1.69,9.0,82.1,-16.0,101.1,2143.0,5.7,635886.0,660813.0,605131.0,571970.0,621035.0,457759.0,608369.0,572041.0,641355.0,605141.0,54.85,0.121,0.127,0.0,1.0,0.0,0.0,2.0,47.0,3.0,Sinker,2.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,Standard,Standard,642715
4,69,69,SI,2020-10-27,100.5,-1.58,6.69,Brusdar Graterol,642715,660813.0,NaN,ball,NaN,NaN,NaN,NaN,11.0,NaN,W,R,R,LAD,TB,B,NaN,NaN,1.0,0.0,2020.0,-1.14,0.87,-0.88,4.94,NaN,NaN,NaN,0.0,7.0,Top,NaN,NaN,NaN,NaN,605131.0,NaN,NaN,4.578675,-146.321570,-1.564144,-17.480085,33.955160,-19.444816,3.50,1.69,NaN,NaN,NaN,99.2,2086.0,5.5,635886.0,660813.0,605131.0,571970.0,621035.0,457759.0,608369.0,572041.0,641355.0,605141.0,55.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.0,2.0,Sinker,2.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,Standard,Standard,642715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [95]:
vet_hitter_statcast['game_date'].dtype

dtype('<M8[ns]')

In [96]:
d = {}

for year, sc_year in vet_hitter_statcast.groupby(vet_hitter_statcast.game_date.dt.year):
    d['df_{}'.format(int(year))] = sc_year


In [97]:
d.keys()

dict_keys(['df_2015', 'df_2016', 'df_2017', 'df_2018', 'df_2019', 'df_2020'])

In [98]:
df_2015 = d['df_2015']
df_2016 = d['df_2016']
df_2017 = d['df_2017']
df_2018 = d['df_2018']
df_2019 = d['df_2019']
df_2020 = d['df_2020']

In [99]:
#df = vet_hitter_statcast[vet_hitter_statcast['events'].notna()]
df = pd.concat([df_2015, df_2016, df_2017, df_2018])

In [100]:
df = df[df['events'].notna()]

In [101]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [102]:
df['pitch_name'].value_counts()

4-Seam Fastball     235353
Slider              112978
2-Seam Fastball      87637
Changeup             77462
Sinker               63135
Curveball            51399
Cutter               38943
Knuckle Curve        16764
Split-Finger         12169
Knuckleball           2815
Intentional Ball      1761
Eephus                 231
Forkball               225
Screwball               30
Pitch Out               29
Unknown                  3
Fastball                 3
Name: pitch_name, dtype: int64

In [103]:
# df['hard'] = 0
# df['soft'] = 0

In [104]:
# def hard_or_soft(pitch):
    
#     hard = ['4-Seam Fastball', '2-Seam Fastball', 'Sinker', 'Cutter', 'Fastball']
#     if pitch 
    


In [105]:
#df.index.to_list()

In [106]:
#df.to_csv('./dummy_statcast_2015-2020.csv')

In [107]:
#df = pd.read_csv('./dummy_statcast_2015-2020.csv')

In [108]:
df.head()

,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,mlb_ID
18572,3084,CH,2015-10-04,88.8,-1.73,5.94,Logan Kensing,571970,434137.0,field_out,hit_into_play,NaN,NaN,NaN,NaN,4.0,Max Muncy flies out to center fielder Brad Mil...,R,L,R,SEA,OAK,X,8.0,fly_ball,0.0,1.0,2015.0,-1.37,0.31,-0.39,2.55,NaN,NaN,NaN,2.0,8.0,Top,152.97,48.79,NaN,NaN,491696.0,NaN,151004_143649,6.299,-129.186,-2.466,-14.744,32.371,-30.811,3.37,1.53,364.0,94.1,30.0,87.9,1757.0,6.2,416075.0,434137.0,491696.0,489149.0,429664.0,572122.0,606466.0,452234.0,543543.0,444432.0,50.0,0.200,0.346,0.0,1.0,0.0,0.0,3.0,61.0,2.0,Changeup,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Strategic,Standard,571970
18574,17758,FF,2015-09-29,94.3,-2.11,5.93,Cam Bedrosian,571970,592135.0,strikeout,swinging_strike,NaN,NaN,NaN,NaN,11.0,Max Muncy strikes out swinging.,R,L,R,LAA,OAK,S,2.0,NaN,1.0,2.0,2015.0,-0.53,1.83,-0.85,3.03,NaN,456714.0,543434.0,2.0,7.0,Top,NaN,NaN,NaN,NaN,542208.0,NaN,150929_212741,4.448,-137.085,-5.965,-5.444,28.396,-10.329,3.37,1.53,NaN,NaN,NaN,94.7,2267.0,6.6,416004.0,592135.0,542208.0,543068.0,543213.0,501896.0,430947.0,461815.0,545361.0,594777.0,50.0,NaN,NaN,0.0,1.0,0.0,0.0,NaN,57.0,4.0,4-Seam Fastball,7.0,1.0,1.0,7.0,1.0,7.0,1.0,7.0,Infield shift,Standard,571970
18578,18811,FT,2015-09-29,90.8,-1.59,6.21,Nick Tropeano,571970,607374.0,home_run,hit_into_play_score,NaN,NaN,NaN,NaN,4.0,Max Muncy homers (3) on a fly ball to center f...,R,L,R,LAA,OAK,X,NaN,fly_ball,3.0,2.0,2015.0,-1.17,1.48,-0.68,2.49,NaN,NaN,NaN,2.0,5.0,Top,114.48,30.29,NaN,NaN,542208.0,NaN,150929_203813,4.944,-132.094,-6.639,-13.380,25.744,-16.191,3.37,1.53,413.0,100.8,29.0,89.1,2075.0,5.3,416004.0,607374.0,542208.0,543068.0,543213.0,501896.0,430947.0,461815.0,545361.0,594777.0,50.0,0.711,1.372,2.0,1.0,0.0,3.0,6.0,37.0,7.0,2-Seam Fastball,6.0,0.0,0.0,6.0,0.0,6.0,0.0,6.0,Infield shift,Standard,571970
18585,19904,CH,2015-09-29,82.1,-1.59,6.35,Nick Tropeano,571970,607374.0,strikeout,called_strike,NaN,NaN,NaN,NaN,6.0,Max Muncy called out on strikes.,R,L,R,LAA,OAK,S,2.0,NaN,2.0,2.0,2015.0,-0.86,0.55,0.69,2.73,NaN,NaN,NaN,0.0,3.0,Top,NaN,NaN,NaN,NaN,542208.0,NaN,150929_194649,6.909,-119.291,-2.345,-7.418,25.842,-28.675,3.37,1.53,NaN,NaN,NaN,79.5,851.0,5.0,416004.0,607374.0,542208.0,543068.0,543213.0,501896.0,430947.0,461815.0,545361.0,594777.0,50.0,NaN,NaN,0.0,1.0,0.0,0.0,NaN,17.0,5.0,Changeup,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,Infield shift,Standard,571970
18590,1537,SI,2015-09-27,88.0,-2.29,5.51,Chris Heston,571970,518790.0,triple,hit_into_play_score,NaN,NaN,NaN,NaN,4.0,Max Muncy triples (1) on a fly ball to left fi...,R,L,R,OAK,SF,X,7.0,fly_ball,3.0,2.0,2015.0,-1.29,0.55,-0.45,2.26,NaN,NaN,502210.0,0.0,6.0,Bot,41.12,84.94,NaN,NaN,623143.0,NaN,150927_145447,7.450,-127.963,-2.810,-14.045,21.256,-27.920,3.36,1.54,359.0,99.2,32.0,88.6,2005.0,6.3,415982.0,518790.0,623143.0,457763.0,605509.0,622110.0,543063.0,592620.0,434636.0,407781.0,50.0,0.422,0.816,1.6,1.0,1.0,2.0,5.0,51.0,6.0,Sinker,2.0,5.0,2.0,5.0,5.0,2.0,2.0,5.0,Infield shift,Standard,571970


In [109]:
print(df['spin_dir'].isna().sum())
print(df['spin_rate_deprecated'].isna().sum())
print(df['break_angle_deprecated'].isna().sum())
print(df['break_length_deprecated'].isna().sum())
print(df['tfs_deprecated'].isna().sum())
print(df['tfs_zulu_deprecated'].isna().sum())
print(df['umpire'].isna().sum())
print(df['sv_id'].isna().sum())

703671
703671
703671
703671
703671
703671
703671
2277


In [110]:
print(len(df))

703671


In [111]:
df.drop(columns=['spin_dir', 'spin_rate_deprecated', 'break_angle_deprecated',
                'break_length_deprecated', 'tfs_deprecated', 'tfs_zulu_deprecated',
                'umpire'], inplace=True)

In [112]:
df.head()

,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,fielder_2,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,mlb_ID
18572,3084,CH,2015-10-04,88.8,-1.73,5.94,Logan Kensing,571970,434137.0,field_out,hit_into_play,4.0,Max Muncy flies out to center fielder Brad Mil...,R,L,R,SEA,OAK,X,8.0,fly_ball,0.0,1.0,2015.0,-1.37,0.31,-0.39,2.55,NaN,NaN,NaN,2.0,8.0,Top,152.97,48.79,491696.0,151004_143649,6.299,-129.186,-2.466,-14.744,32.371,-30.811,3.37,1.53,364.0,94.1,30.0,87.9,1757.0,6.2,416075.0,434137.0,491696.0,489149.0,429664.0,572122.0,606466.0,452234.0,543543.0,444432.0,50.0,0.200,0.346,0.0,1.0,0.0,0.0,3.0,61.0,2.0,Changeup,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Strategic,Standard,571970
18574,17758,FF,2015-09-29,94.3,-2.11,5.93,Cam Bedrosian,571970,592135.0,strikeout,swinging_strike,11.0,Max Muncy strikes out swinging.,R,L,R,LAA,OAK,S,2.0,NaN,1.0,2.0,2015.0,-0.53,1.83,-0.85,3.03,NaN,456714.0,543434.0,2.0,7.0,Top,NaN,NaN,542208.0,150929_212741,4.448,-137.085,-5.965,-5.444,28.396,-10.329,3.37,1.53,NaN,NaN,NaN,94.7,2267.0,6.6,416004.0,592135.0,542208.0,543068.0,543213.0,501896.0,430947.0,461815.0,545361.0,594777.0,50.0,NaN,NaN,0.0,1.0,0.0,0.0,NaN,57.0,4.0,4-Seam Fastball,7.0,1.0,1.0,7.0,1.0,7.0,1.0,7.0,Infield shift,Standard,571970
18578,18811,FT,2015-09-29,90.8,-1.59,6.21,Nick Tropeano,571970,607374.0,home_run,hit_into_play_score,4.0,Max Muncy homers (3) on a fly ball to center f...,R,L,R,LAA,OAK,X,NaN,fly_ball,3.0,2.0,2015.0,-1.17,1.48,-0.68,2.49,NaN,NaN,NaN,2.0,5.0,Top,114.48,30.29,542208.0,150929_203813,4.944,-132.094,-6.639,-13.380,25.744,-16.191,3.37,1.53,413.0,100.8,29.0,89.1,2075.0,5.3,416004.0,607374.0,542208.0,543068.0,543213.0,501896.0,430947.0,461815.0,545361.0,594777.0,50.0,0.711,1.372,2.0,1.0,0.0,3.0,6.0,37.0,7.0,2-Seam Fastball,6.0,0.0,0.0,6.0,0.0,6.0,0.0,6.0,Infield shift,Standard,571970
18585,19904,CH,2015-09-29,82.1,-1.59,6.35,Nick Tropeano,571970,607374.0,strikeout,called_strike,6.0,Max Muncy called out on strikes.,R,L,R,LAA,OAK,S,2.0,NaN,2.0,2.0,2015.0,-0.86,0.55,0.69,2.73,NaN,NaN,NaN,0.0,3.0,Top,NaN,NaN,542208.0,150929_194649,6.909,-119.291,-2.345,-7.418,25.842,-28.675,3.37,1.53,NaN,NaN,NaN,79.5,851.0,5.0,416004.0,607374.0,542208.0,543068.0,543213.0,501896.0,430947.0,461815.0,545361.0,594777.0,50.0,NaN,NaN,0.0,1.0,0.0,0.0,NaN,17.0,5.0,Changeup,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,Infield shift,Standard,571970
18590,1537,SI,2015-09-27,88.0,-2.29,5.51,Chris Heston,571970,518790.0,triple,hit_into_play_score,4.0,Max Muncy triples (1) on a fly ball to left fi...,R,L,R,OAK,SF,X,7.0,fly_ball,3.0,2.0,2015.0,-1.29,0.55,-0.45,2.26,NaN,NaN,502210.0,0.0,6.0,Bot,41.12,84.94,623143.0,150927_145447,7.450,-127.963,-2.810,-14.045,21.256,-27.920,3.36,1.54,359.0,99.2,32.0,88.6,2005.0,6.3,415982.0,518790.0,623143.0,457763.0,605509.0,622110.0,543063.0,592620.0,434636.0,407781.0,50.0,0.422,0.816,1.6,1.0,1.0,2.0,5.0,51.0,6.0,Sinker,2.0,5.0,2.0,5.0,5.0,2.0,2.0,5.0,Infield shift,Standard,571970


In [113]:
df['babip_value'].value_counts()

0.0    563398
1.0    140273
Name: babip_value, dtype: int64

In [114]:
df['iso_value'].value_counts()

0.0    646195
1.0     32218
3.0     21925
2.0      3333
Name: iso_value, dtype: int64

In [115]:
df['woba_value'].value_counts()

0.00    473175
0.90    110802
0.70     62218
1.25     32218
2.00     21925
1.60      3333
Name: woba_value, dtype: int64

wOBA = (0.690×uBB + 0.722×HBP + 0.888×1B + 1.271×2B + 1.616×3B +
2.101×HR) / (AB + BB – IBB + SF + HBP)

In [116]:
interesting_columns = ['batter', 'zone', 'game_type', 'stand', 'p_throws', 'hit_location', 'events'
                  'bb_type', 'balls', 'strikes', 'on_3b', 'on_2b', 'on_1b',
                  'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
                  'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
                  'release_spin_rate', 'estimated_ba_using_speedangle', 
                   'estimated_woba_using_speedangle', 'woba_value','babip_value', 
                   'iso_value', 'launch_speed_angle', 'pitch_number', 'hard', 'soft']

In [117]:
hitting_cols = ['batter', 'zone', 'game_type', 'stand', 'p_throws', 'hit_location', 
                'events','bb_type', 'balls', 'strikes', 'on_3b', 'on_2b', 'on_1b',
                'outs_when_up', 'inning', 'hc_x', 'hc_y','hit_distance_sc', 
                'launch_speed', 'launch_angle', 'estimated_ba_using_speedangle', 
                'woba_value','babip_value', 'iso_value', 'launch_speed_angle', 
                'pitch_number', 'pitch_name', 'effective_speed','if_fielding_alignment', 
                'of_fielding_alignment', 'bat_score', 'fld_score']

In [118]:
sc_hitting = ['batter', 'game_type', 'p_throws', 'bb_type', 
            'hc_x', 'hc_y','hit_distance_sc', 'launch_speed', 'launch_angle', 
            'woba_value','babip_value', 'iso_value', 'launch_speed_angle', 'pitch_name']

**Curious about ISO value**

In [119]:
hitting_df = df[sc_hitting]

In [120]:
#hitting_df = pd.get_dummies(data=hitting_df, columns=['events'])

In [121]:
hard = ['4-Seam Fastball', '2-Seam Fastball', 'Sinker', 'Cutter', 'Fastball']

# for index in df.index.to_list():
#     #print(index)
#     if df.loc[index, 'pitch_name'] in hard:
#         df.loc[index, 'hard'] = 1
#     else:
#          df.loc[index, 'soft'] = 1
            
hitting_df['hard'] = [1 if x in hard else 0 for x in hitting_df['pitch_name']]
#hitting_df['soft'] = [1 if x not in hard else 0 for x in hitting_df['pitch_name']]
hitting_df.drop(columns=['pitch_name'], inplace=True)

<ipython-input-121-e0deca61c74b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hitting_df['hard'] = [1 if x in hard else 0 for x in hitting_df['pitch_name']]
/Users/johnnybex/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Fill in players on base with 1 instead of player_id, null is 0


In [122]:
hitting_df = pd.get_dummies(data=hitting_df, columns=['bb_type'], drop_first=True)

In [123]:
hitting_df = pd.get_dummies(data=hitting_df, columns=['launch_speed_angle'], drop_first=True)

In [124]:
# hitting_df['winning'] = hitting_df['bat_score'] - hitting_df['fld_score']
# hitting_df['winning'] = ['losing' if x <0 else 'tied' if x==0 else 'winning' for x in hitting_df['winning']]
# hitting_df = pd.get_dummies(data=hitting_df, columns=['winning'], drop_first=True)
# hitting_df = hitting_df.drop(columns=['bat_score', 'fld_score'])

In [125]:
# Separating Regualar season

hitting_df['regular_season'] = [1 if x == 'R' else 0 for x in df['game_type']]

In [126]:
# hitting_df = pd.get_dummies(data = hitting_df, columns = ['if_fielding_alignment', 'of_fielding_alignment'], 
#                     drop_first=True)

In [127]:
# Separating postseason

postseason = ['F', 'D', 'L', 'W']

hitting_df['postseason'] = [1 if x in postseason else 0 for x in df['game_type']]

In [128]:
# Binarizing hitting handedness

#hitting_df = pd.get_dummies(data=hitting_df, columns=['stand'], drop_first=True)

#df['stance'] = [1 if x=='R' else 0 for x in df['stand']]

In [129]:
hitting_df = pd.get_dummies(data=hitting_df, columns=['p_throws'], drop_first=True)

In [130]:
#df['half_inning'] = [0 if x == 'Top' else 1 for x in df['inning_topbot']]

In [131]:
# hitting_df[['on_1b', 'on_2b', 'on_3b']].fillna(0)

# hitting_df['on_1b'] = [1 if x > 0 else 0 for x in df['on_1b']]
# hitting_df['on_2b'] = [1 if x > 0 else 0 for x in df['on_2b']]
# hitting_df['on_3b'] = [1 if x > 0 else 0 for x in df['on_3b']]

In [132]:
regular_hitting = hitting_df[hitting_df['game_type'] == 'R']
regular_hitting = regular_hitting.drop(columns=['postseason', 'regular_season', 'game_type'])

In [133]:
regular_hitting.to_csv('./regular_season_sc_hitting')

In [134]:
regular_hitting

,batter,hc_x,hc_y,hit_distance_sc,launch_speed,launch_angle,woba_value,babip_value,iso_value,hard,bb_type_ground_ball,bb_type_line_drive,bb_type_popup,launch_speed_angle_2.0,launch_speed_angle_3.0,launch_speed_angle_4.0,launch_speed_angle_5.0,launch_speed_angle_6.0,p_throws_R
18572,571970,152.97,48.79,364.0,94.1,30.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,1
18574,571970,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,1
18578,571970,114.48,30.29,413.0,100.8,29.0,2.0,0.0,3.0,1,0,0,0,0,0,0,0,1,1
18585,571970,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,1
18590,571970,41.12,84.94,359.0,99.2,32.0,1.6,1.0,2.0,1,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3359494,471083,182.28,110.40,266.0,82.5,23.0,0.0,0.0,0.0,1,0,1,0,0,1,0,0,0,1
3359495,471083,92.07,71.51,310.0,86.4,31.0,0.0,0.0,0.0,1,0,0,0,0,1,0,0,0,1
3359497,471083,107.93,154.86,73.0,75.3,6.0,0.0,0.0,0.0,1,1,0,0,1,0,0,0,0,0
3359498,471083,129.13,183.33,11.0,94.0,-13.0,0.0,0.0,0.0,1,1,0,0,1,0,0,0,0,1


In [135]:
rh_gb = regular_hitting.groupby('batter').mean()

In [136]:
rh_gb

,hc_x,hc_y,hit_distance_sc,launch_speed,launch_angle,woba_value,babip_value,iso_value,hard,bb_type_ground_ball,bb_type_line_drive,bb_type_popup,launch_speed_angle_2.0,launch_speed_angle_3.0,launch_speed_angle_4.0,launch_speed_angle_5.0,launch_speed_angle_6.0,p_throws_R
batter,,,,,,,,,,,,,,,,,,
116338,125.443848,127.028242,162.284182,88.406295,10.786925,0.309315,0.181019,0.154657,0.620387,0.365554,0.154657,0.052724,0.288225,0.181019,0.159930,0.038664,0.036907,0.695958
120074,140.790445,112.335695,205.639136,93.089468,16.281993,0.396011,0.191781,0.251410,0.622885,0.258662,0.209508,0.061241,0.186946,0.186140,0.165995,0.059629,0.103948,0.728445
121347,121.053223,120.489121,174.983903,90.854797,11.721402,0.332737,0.154734,0.181293,0.650115,0.279446,0.153580,0.043880,0.220554,0.154734,0.130485,0.046189,0.062356,0.680139
133380,123.462282,119.972500,184.731183,87.362562,15.761084,0.308607,0.193424,0.162476,0.647969,0.299807,0.232108,0.069632,0.234043,0.241779,0.185687,0.052224,0.044487,0.762089
134181,124.160108,119.199850,181.657124,89.018396,13.367310,0.365899,0.225458,0.172851,0.637388,0.325035,0.222170,0.058713,0.248473,0.201503,0.205261,0.052607,0.051667,0.721935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664057,128.141053,131.789053,139.686047,87.140230,5.034483,0.252632,0.177632,0.046053,0.565789,0.348684,0.164474,0.013158,0.230263,0.078947,0.184211,0.039474,0.013158,0.855263
664058,120.845906,127.307756,161.290598,84.000000,13.404000,0.303247,0.250000,0.081169,0.597403,0.340909,0.253247,0.074675,0.275974,0.233766,0.256494,0.019481,0.003247,0.610390
664068,123.722115,121.515740,185.387755,86.171118,17.394410,0.267593,0.193416,0.104938,0.553498,0.246914,0.181070,0.053498,0.185185,0.205761,0.174897,0.032922,0.034979,0.732510


In [137]:
# rh_gb = regular_hitting.groupby(by=['batter', 'hard', 'zone','outs_when_up', 'inning', 
#                             'balls', 'strikes'], as_index=False).mean()

In [138]:
# rh_gb.fillna(0, inplace=True)

In [139]:
rh_gb_sc = StandardScaler().fit_transform(rh_gb)

In [149]:
model = KMeans(n_clusters=3, random_state=0)
model.fit(rh_gb_sc)

KMeans(n_clusters=3, random_state=0)

In [150]:
labels = model.labels_

rh_gb['labels'] = labels
rh_gb.head()

,hc_x,hc_y,hit_distance_sc,launch_speed,launch_angle,woba_value,babip_value,iso_value,hard,bb_type_ground_ball,bb_type_line_drive,bb_type_popup,launch_speed_angle_2.0,launch_speed_angle_3.0,launch_speed_angle_4.0,launch_speed_angle_5.0,launch_speed_angle_6.0,p_throws_R,labels
batter,,,,,,,,,,,,,,,,,,,
116338,125.443848,127.028242,162.284182,88.406295,10.786925,0.309315,0.181019,0.154657,0.620387,0.365554,0.154657,0.052724,0.288225,0.181019,0.159930,0.038664,0.036907,0.695958,2
120074,140.790445,112.335695,205.639136,93.089468,16.281993,0.396011,0.191781,0.251410,0.622885,0.258662,0.209508,0.061241,0.186946,0.186140,0.165995,0.059629,0.103948,0.728445,0
121347,121.053223,120.489121,174.983903,90.854797,11.721402,0.332737,0.154734,0.181293,0.650115,0.279446,0.153580,0.043880,0.220554,0.154734,0.130485,0.046189,0.062356,0.680139,0
133380,123.462282,119.972500,184.731183,87.362562,15.761084,0.308607,0.193424,0.162476,0.647969,0.299807,0.232108,0.069632,0.234043,0.241779,0.185687,0.052224,0.044487,0.762089,0
134181,124.160108,119.199850,181.657124,89.018396,13.367310,0.365899,0.225458,0.172851,0.637388,0.325035,0.222170,0.058713,0.248473,0.201503,0.205261,0.052607,0.051667,0.721935,0


In [151]:
score = silhouette_score(rh_gb, labels, metric='euclidean')
score

0.29502481834206346

In [152]:
data = batting_stats_range('2015-04-01', '2018-11-07')

data = data[data['G'] > 100]

data.head()

,Name,Age,#days,Lev,Tm,G,PA,AB,R,H,2B,3B,HR,RBI,BB,IBB,SO,HBP,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS,mlb_ID
3,José Abreu,31,893,MLB-AL,Chicago,597,2591,2357,318,682,145,11,110,381,158,31,493,56,0,20,72,5,0,0.289,0.346,0.500,0.846,547989
4,Dustin Ackley,28,1733,MLB-AL,"New York,Seattle",125,334,299,34,64,11,3,10,34,26,0,54,1,3,5,3,2,2,0.214,0.275,0.371,0.646,554429
5,Ronald Acuna Jr.,20,879,MLB-NL,Atlanta,111,487,433,78,127,26,4,26,64,45,2,123,6,0,3,4,16,4,0.293,0.366,0.552,0.917,660670
6,Cristhian Adames,25,1399,MLB-NL,Colorado,152,328,291,30,62,8,4,2,20,28,1,64,5,4,0,5,2,3,0.213,0.293,0.289,0.582,542436
9,Lane Adams,28,879,MLB-NL,Atlanta,108,151,134,29,36,5,1,7,26,14,0,45,1,1,1,4,11,0,0.269,0.340,0.478,0.818,572669


In [153]:
group_0 = rh_gb[rh_gb['labels'] == 0]

group_1 = rh_gb[rh_gb['labels'] == 1]

group_2 = rh_gb[rh_gb['labels'] == 2]

group_3 = rh_gb[rh_gb['labels'] == 3]

In [158]:
pd.merge(group_0, data, left_on='batter', right_on='mlb_ID')

,hc_x,hc_y,hit_distance_sc,launch_speed,launch_angle,woba_value,babip_value,iso_value,hard,bb_type_ground_ball,bb_type_line_drive,bb_type_popup,launch_speed_angle_2.0,launch_speed_angle_3.0,launch_speed_angle_4.0,launch_speed_angle_5.0,launch_speed_angle_6.0,p_throws_R,labels,Name,Age,#days,Lev,Tm,G,PA,AB,R,H,2B,3B,HR,RBI,BB,IBB,SO,HBP,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS,mlb_ID
0,140.790445,112.335695,205.639136,93.089468,16.281993,0.396011,0.191781,0.251410,0.622885,0.258662,0.209508,0.061241,0.186946,0.186140,0.165995,0.059629,0.103948,0.728445,0,David Ortiz,40,1607,MLB-AL,Boston,297,1239,1064,152,313,85,1,75,235,157,31,180,2,0,16,38,2,1,0.294,0.381,0.587,0.968,120074
1,121.053223,120.489121,174.983903,90.854797,11.721402,0.332737,0.154734,0.181293,0.650115,0.279446,0.153580,0.043880,0.220554,0.154734,0.130485,0.046189,0.062356,0.680139,0,Alex Rodriguez,40,1658,MLB-AL,New York,216,863,748,102,176,29,1,42,117,98,5,212,7,0,10,21,7,0,0.235,0.326,0.445,0.771,121347
2,123.462282,119.972500,184.731183,87.362562,15.761084,0.308607,0.193424,0.162476,0.647969,0.299807,0.232108,0.069632,0.234043,0.241779,0.185687,0.052224,0.044487,0.762089,0,Aramis Ramirez,37,1972,MLB-NL,"Pittsburgh,Milwaukee",137,516,475,43,117,31,1,17,75,31,3,68,5,0,5,23,1,0,0.246,0.297,0.423,0.720,133380
3,124.160108,119.199850,181.657124,89.018396,13.367310,0.365899,0.225458,0.172851,0.637388,0.325035,0.222170,0.058713,0.248473,0.201503,0.205261,0.052607,0.051667,0.721935,0,Adrian Beltre,39,879,MLB-AL,Texas,509,2129,1923,268,562,108,7,82,323,162,14,279,19,0,25,48,4,0,0.292,0.349,0.484,0.833,134181
4,130.831298,119.803543,182.181650,89.447312,13.723739,0.338589,0.209202,0.174233,0.586503,0.308589,0.195092,0.065644,0.240491,0.208589,0.193865,0.041718,0.043558,0.727607,0,Carlos Beltran,40,1243,MLB-AL,"New York,Texas,Houston",413,1633,1497,190,403,96,1,62,211,113,9,288,7,0,16,40,1,0,0.269,0.320,0.459,0.779,136860
5,123.016267,119.978836,187.299728,90.227561,14.841463,0.321821,0.168760,0.146782,0.616954,0.255887,0.176609,0.057300,0.191523,0.189168,0.153061,0.045526,0.050235,0.780220,0,Jayson Werth,38,1243,MLB-NL,Washington,301,1273,1108,170,258,54,2,43,140,144,1,292,8,0,13,29,9,3,0.233,0.322,0.402,0.724,150029
6,131.132398,119.434975,182.100749,88.594583,14.466376,0.313735,0.210319,0.122850,0.627027,0.324816,0.211302,0.055037,0.231450,0.212776,0.220639,0.055037,0.045700,0.757248,0,Victor Martinez,39,887,MLB-AL,Detroit,514,2038,1852,174,485,79,0,57,251,149,23,254,18,0,19,71,0,0,0.262,0.320,0.397,0.717,400121
7,118.365036,121.023116,180.668585,90.314640,13.296794,0.320516,0.184275,0.170762,0.633497,0.347666,0.195332,0.064292,0.263309,0.210483,0.189599,0.055283,0.058149,0.757166,0,Albert Pujols,38,912,MLB-AL,Los Angeles,575,2445,2253,259,563,78,0,113,379,164,24,305,12,0,16,77,13,2,0.250,0.302,0.435,0.737,405395
8,128.653632,121.699372,179.070000,86.447307,13.133489,0.330386,0.190193,0.185736,0.585438,0.279346,0.199108,0.032689,0.188707,0.170877,0.154532,0.035661,0.053492,0.728083,0,Marlon Byrd,38,1731,"MLB-NL,MLB-AL","San Francisco,Cincinnati,Cleveland",168,673,621,69,156,31,5,28,92,40,3,183,4,0,8,11,2,1,0.251,0.297,0.452,0.750,407781
9,129.454293,124.155152,164.855093,91.309936,9.765605,0.348368,0.179079,0.163180,0.641841,0.327197,0.151464,0.048536,0.240167,0.142259,0.161506,0.040167,0.052720,0.710460,0,Matt Holliday,38,878,"MLB-NL,MLB-AL","St. Louis,Colorado,New York",312,1195,1037,125,259,56,2,45,164,132,6,252,17,0,9,32,3,0,0.250,0.341,0.438,0.779,407812


In [155]:
pd.merge(group_1, data, left_on='batter', right_on='mlb_ID')

,hc_x,hc_y,hit_distance_sc,launch_speed,launch_angle,woba_value,babip_value,iso_value,hard,bb_type_ground_ball,bb_type_line_drive,bb_type_popup,launch_speed_angle_2.0,launch_speed_angle_3.0,launch_speed_angle_4.0,launch_speed_angle_5.0,launch_speed_angle_6.0,p_throws_R,labels,Name,Age,#days,Lev,Tm,G,PA,AB,R,H,2B,3B,HR,RBI,BB,IBB,SO,HBP,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS,mlb_ID
0,126.783293,145.826886,115.450000,88.776119,0.925373,0.136594,0.083333,0.043478,0.753623,0.442029,0.090580,0.014493,0.278986,0.068841,0.090580,0.014493,0.025362,0.757246,1,Jon Lester,34,882,MLB-NL,Chicago,118,276,234,18,25,6,0,2,21,13,0,96,0,28,1,4,1,0,0.107,0.153,0.158,0.311,452657
1,130.381147,146.760596,95.564706,83.356425,-2.754190,0.203994,0.172524,0.019169,0.680511,0.511182,0.127796,0.015974,0.316294,0.063898,0.146965,0.006390,0.006390,0.773163,1,Max Scherzer,33,884,MLB-NL,Washington,127,312,271,20,55,3,0,1,23,9,0,83,2,30,0,2,1,0,0.203,0.234,0.225,0.459,453286
2,138.473571,139.117269,120.064356,85.281333,3.128889,0.273620,0.168712,0.085890,0.702454,0.453988,0.153374,0.046012,0.365031,0.122699,0.156442,0.012270,0.009202,0.831288,1,Sam Fuld,33,1971,MLB-AL,Oakland,108,325,290,34,57,16,3,2,22,30,1,55,2,2,1,6,9,3,0.197,0.276,0.293,0.569,453539
3,126.035448,150.851269,129.703297,78.873000,12.420000,0.115918,0.085714,0.016327,0.632653,0.322449,0.102041,0.053061,0.142857,0.126531,0.110204,0.000000,0.000000,0.800000,1,Gio Gonzalez,32,879,MLB-NL,"Washington,Milwaukee",115,245,212,8,21,4,0,0,10,6,0,102,3,23,1,1,0,0,0.099,0.135,0.118,0.253,461829
4,127.365088,141.677632,145.566265,85.420408,10.051020,0.196784,0.152047,0.029240,0.631579,0.374269,0.140351,0.052632,0.239766,0.140351,0.169591,0.005848,0.005848,0.719298,1,Emilio Bonifacio,32,1366,"MLB-NL,MLB-AL","Atlanta,Chicago",107,169,154,13,26,3,1,0,10,6,0,48,1,7,1,2,2,4,0.169,0.204,0.201,0.405,466988
5,133.597757,158.843178,79.971429,84.525316,-0.177215,0.100235,0.093897,0.004695,0.591549,0.384977,0.065728,0.037559,0.234742,0.061033,0.065728,0.000000,0.000000,0.760563,1,Chase Anderson,30,891,MLB-NL,"Arizona,Milwaukee",106,213,190,5,20,1,0,0,9,2,0,102,1,20,0,3,0,0,0.105,0.119,0.111,0.230,502624
6,129.826030,149.659950,127.847826,83.704138,3.958621,0.145686,0.113725,0.015686,0.709804,0.533333,0.125490,0.027451,0.298039,0.109804,0.149020,0.007843,0.000000,0.701961,1,Julio Teheran,27,882,MLB-NL,Atlanta,122,255,198,7,30,1,0,1,14,5,0,50,1,49,2,1,0,0,0.152,0.175,0.172,0.346,527054
7,129.838788,134.229697,121.737864,84.518807,3.821101,0.273171,0.182927,0.067073,0.649390,0.442073,0.134146,0.024390,0.335366,0.109756,0.155488,0.006098,0.012195,0.667683,1,Cristhian Adames,25,1399,MLB-NL,Colorado,152,328,291,30,62,8,4,2,20,28,1,64,5,4,0,5,2,3,0.213,0.293,0.289,0.582,542436
8,132.810379,154.905909,106.096386,83.152941,3.725490,0.094177,0.080321,0.012048,0.718876,0.361446,0.076305,0.052209,0.212851,0.092369,0.084337,0.004016,0.000000,0.751004,1,Kyle Hendricks,28,881,MLB-NL,Chicago,114,249,228,7,20,3,0,0,9,4,0,112,1,16,0,2,0,0,0.088,0.107,0.101,0.208,543294
9,129.122685,145.127584,107.560976,82.574803,2.614173,0.168103,0.125000,0.034483,0.655172,0.456897,0.068966,0.021552,0.301724,0.120690,0.086207,0.000000,0.004310,0.780172,1,Tanner Roark,31,893,MLB-NL,Washington,107,232,204,9,29,6,1,0,11,7,0,75,1,20,0,2,0,0,0.142,0.175,0.181,0.356,543699


In [156]:
pd.merge(group_2, data, left_on='batter', right_on='mlb_ID')

,hc_x,hc_y,hit_distance_sc,launch_speed,launch_angle,woba_value,babip_value,iso_value,hard,bb_type_ground_ball,bb_type_line_drive,bb_type_popup,launch_speed_angle_2.0,launch_speed_angle_3.0,launch_speed_angle_4.0,launch_speed_angle_5.0,launch_speed_angle_6.0,p_throws_R,labels,Name,Age,#days,Lev,Tm,G,PA,AB,R,H,2B,3B,HR,RBI,BB,IBB,SO,HBP,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS,mlb_ID
0,125.443848,127.028242,162.284182,88.406295,10.786925,0.309315,0.181019,0.154657,0.620387,0.365554,0.154657,0.052724,0.288225,0.181019,0.159930,0.038664,0.036907,0.695958,2,Torii Hunter,39,1972,MLB-AL,Minnesota,139,567,521,67,125,22,0,22,81,35,1,105,6,0,5,14,2,2,0.240,0.293,0.409,0.702,116338
1,126.693322,128.348288,159.035573,87.498571,7.785714,0.325980,0.215686,0.122549,0.666667,0.360294,0.181373,0.044118,0.274510,0.142157,0.171569,0.044118,0.031863,0.725490,2,Michael Cuddyer,36,1972,MLB-NL,New York,117,408,379,44,98,18,1,10,41,24,0,88,4,0,1,13,2,0,0.259,0.309,0.391,0.699,150212
2,133.160286,128.649529,156.731203,86.213652,9.293515,0.305971,0.237410,0.106475,0.608633,0.424460,0.243165,0.043165,0.343885,0.169784,0.224460,0.043165,0.024460,0.815827,2,A.J. Pierzynski,39,1629,MLB-NL,Atlanta,193,695,654,53,176,39,1,11,72,25,3,66,10,0,6,32,1,1,0.269,0.304,0.382,0.686,150229
3,136.948891,127.859909,168.240166,85.568821,15.300380,0.285539,0.182810,0.115962,0.658936,0.320600,0.170532,0.073670,0.244202,0.229195,0.177353,0.028649,0.017735,0.802183,2,Jimmy Rollins,37,1723,"MLB-AL,MLB-NL","Chicago,Los Angeles",185,729,666,96,149,32,4,15,49,60,0,119,0,1,2,14,17,6,0.224,0.287,0.351,0.638,276519
4,125.322622,125.574355,166.527981,88.473077,10.373932,0.302744,0.181402,0.135671,0.582317,0.333841,0.176829,0.060976,0.262195,0.169207,0.184451,0.033537,0.045732,0.717988,2,Juan Uribe,37,1671,"MLB-NL,MLB-AL","Atlanta,Los Angeles,New York,Cleveland",190,655,597,59,140,26,0,21,68,49,4,128,5,0,4,18,2,0,0.235,0.296,0.384,0.680,346874
5,126.182249,136.548272,128.737120,82.912107,5.226392,0.285300,0.224719,0.058989,0.642322,0.441011,0.191948,0.044007,0.370787,0.161985,0.180712,0.011236,0.004682,0.785581,2,Ichiro Suzuki,44,1030,"MLB-AL,MLB-NL","Seattle,Miami",443,1065,965,117,245,26,11,5,63,81,3,135,4,9,6,14,22,6,0.254,0.313,0.319,0.632,400085
6,135.490972,122.285995,176.574018,87.704093,12.752558,0.306765,0.186275,0.124183,0.643137,0.320261,0.187582,0.042484,0.226797,0.191503,0.195425,0.044444,0.026144,0.831373,2,Chase Utley,39,879,MLB-NL,"Los Angeles,Philadelphia",453,1528,1358,177,316,77,10,31,139,121,7,270,35,2,12,13,15,1,0.233,0.309,0.373,0.682,400284
7,118.441681,123.882090,149.976587,90.236753,4.698710,0.337537,0.231158,0.099789,0.626526,0.393263,0.213474,0.011789,0.294316,0.096842,0.222316,0.051789,0.034526,0.737263,2,Joe Mauer,35,879,MLB-AL,Minnesota,557,2382,2097,270,583,119,8,34,234,263,30,374,7,2,13,59,6,1,0.278,0.358,0.391,0.749,408045
8,130.580777,122.165777,162.494340,89.624221,9.605536,0.339152,0.239401,0.147132,0.571072,0.356608,0.216958,0.037406,0.301746,0.154613,0.162095,0.052369,0.034913,0.805486,2,Justin Morneau,35,1607,"MLB-NL,MLB-AL","Colorado,Chicago",106,400,371,35,105,24,4,9,40,25,3,77,2,0,2,7,0,0,0.283,0.330,0.442,0.772,408047
9,128.755545,128.417890,153.101545,85.727309,9.635207,0.329500,0.251223,0.112017,0.619358,0.399130,0.244698,0.054921,0.317020,0.188146,0.231648,0.028820,0.020663,0.766721,2,Brandon Phillips,37,879,"MLB-NL,MLB-AL","Atlanta,Cincinnati,Los Angeles,Boston",442,1838,1733,228,499,87,4,37,196,70,5,216,20,5,10,52,48,18,0.288,0.321,0.407,0.728,408252


In [148]:
pd.merge(group_3, data, left_on='batter', right_on='mlb_ID')

,hc_x,hc_y,hit_distance_sc,launch_speed,launch_angle,woba_value,babip_value,iso_value,hard,bb_type_ground_ball,bb_type_line_drive,bb_type_popup,launch_speed_angle_2.0,launch_speed_angle_3.0,launch_speed_angle_4.0,launch_speed_angle_5.0,launch_speed_angle_6.0,p_throws_R,labels,Name,Age,#days,Lev,Tm,G,PA,AB,R,H,2B,3B,HR,RBI,BB,IBB,SO,HBP,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS,mlb_ID
0,126.783293,145.826886,115.450000,88.776119,0.925373,0.136594,0.083333,0.043478,0.753623,0.442029,0.090580,0.014493,0.278986,0.068841,0.090580,0.014493,0.025362,0.757246,3,Jon Lester,34,882,MLB-NL,Chicago,118,276,234,18,25,6,0,2,21,13,0,96,0,28,1,4,1,0,0.107,0.153,0.158,0.311,452657
1,130.381147,146.760596,95.564706,83.356425,-2.754190,0.203994,0.172524,0.019169,0.680511,0.511182,0.127796,0.015974,0.316294,0.063898,0.146965,0.006390,0.006390,0.773163,3,Max Scherzer,33,884,MLB-NL,Washington,127,312,271,20,55,3,0,1,23,9,0,83,2,30,0,2,1,0,0.203,0.234,0.225,0.459,453286
2,138.473571,139.117269,120.064356,85.281333,3.128889,0.273620,0.168712,0.085890,0.702454,0.453988,0.153374,0.046012,0.365031,0.122699,0.156442,0.012270,0.009202,0.831288,3,Sam Fuld,33,1971,MLB-AL,Oakland,108,325,290,34,57,16,3,2,22,30,1,55,2,2,1,6,9,3,0.197,0.276,0.293,0.569,453539
3,126.035448,150.851269,129.703297,78.873000,12.420000,0.115918,0.085714,0.016327,0.632653,0.322449,0.102041,0.053061,0.142857,0.126531,0.110204,0.000000,0.000000,0.800000,3,Gio Gonzalez,32,879,MLB-NL,"Washington,Milwaukee",115,245,212,8,21,4,0,0,10,6,0,102,3,23,1,1,0,0,0.099,0.135,0.118,0.253,461829
4,127.365088,141.677632,145.566265,85.420408,10.051020,0.196784,0.152047,0.029240,0.631579,0.374269,0.140351,0.052632,0.239766,0.140351,0.169591,0.005848,0.005848,0.719298,3,Emilio Bonifacio,32,1366,"MLB-NL,MLB-AL","Atlanta,Chicago",107,169,154,13,26,3,1,0,10,6,0,48,1,7,1,2,2,4,0.169,0.204,0.201,0.405,466988
5,133.597757,158.843178,79.971429,84.525316,-0.177215,0.100235,0.093897,0.004695,0.591549,0.384977,0.065728,0.037559,0.234742,0.061033,0.065728,0.000000,0.000000,0.760563,3,Chase Anderson,30,891,MLB-NL,"Arizona,Milwaukee",106,213,190,5,20,1,0,0,9,2,0,102,1,20,0,3,0,0,0.105,0.119,0.111,0.230,502624
6,129.826030,149.659950,127.847826,83.704138,3.958621,0.145686,0.113725,0.015686,0.709804,0.533333,0.125490,0.027451,0.298039,0.109804,0.149020,0.007843,0.000000,0.701961,3,Julio Teheran,27,882,MLB-NL,Atlanta,122,255,198,7,30,1,0,1,14,5,0,50,1,49,2,1,0,0,0.152,0.175,0.172,0.346,527054
7,129.838788,134.229697,121.737864,84.518807,3.821101,0.273171,0.182927,0.067073,0.649390,0.442073,0.134146,0.024390,0.335366,0.109756,0.155488,0.006098,0.012195,0.667683,3,Cristhian Adames,25,1399,MLB-NL,Colorado,152,328,291,30,62,8,4,2,20,28,1,64,5,4,0,5,2,3,0.213,0.293,0.289,0.582,542436
8,132.810379,154.905909,106.096386,83.152941,3.725490,0.094177,0.080321,0.012048,0.718876,0.361446,0.076305,0.052209,0.212851,0.092369,0.084337,0.004016,0.000000,0.751004,3,Kyle Hendricks,28,881,MLB-NL,Chicago,114,249,228,7,20,3,0,0,9,4,0,112,1,16,0,2,0,0,0.088,0.107,0.101,0.208,543294
9,129.122685,145.127584,107.560976,82.574803,2.614173,0.168103,0.125000,0.034483,0.655172,0.456897,0.068966,0.021552,0.301724,0.120690,0.086207,0.000000,0.004310,0.780172,3,Tanner Roark,31,893,MLB-NL,Washington,107,232,204,9,29,6,1,0,11,7,0,75,1,20,0,2,0,0,0.142,0.175,0.181,0.356,543699
